In [ ]:
"""Combining Front Office and Middle Office

Demonstration of how to give front office users an view of intraday trading activity on top of their middle office IBOR.

Attributes
----------
instruments
transactions
sub-holdings keys
properties
orders
derived portfolios
cocoon
"""

# Combining Front Office and Middle Office

In this example we demonstrate how to give front office users a view of intraday trading activity on top of their middle office IBOR. We set up a portfolio with multiple strategies and generate orders which are booked back into a [derived portfolio](https://support.finbourne.com/what-is-a-derived-portfolio) showing updated positions in the strategies.

To illustrate, below we will BUY and SELL <i>Amazon, Inc.</i> stock across two strategies, and show an updated view of the portfolio intraday. Consider the <b>Tech</b> strategy. At SOD we have 5000 units, we SELL 500 units, leaving 4500 in the portfolio. 

<br>


|Portfolio Code | Instrument Name |ISIN| Strategy  | SOD Holdings | Intraday orders | Intraday Holdings|
|:-------------:| :-------------: | :-----------------: |:-------------: | :-------------: | :-------------: | :-------------: |
|us_long_live| Amazon |US0231351067 |Balanced | 0 | BUY of 4000 units | <font color="red">4000 </font>|
|us_long_live| Amazon |US0231351067|Tech | 5000 | SELL of 500 units | <font color="red">4500 </font>|

<br>

## Imports

In [1]:
import lusid
import lusid.models as models
import lusid.api as la
import lusid.models as lm
from lusid import ApiException
from lusid.utilities import ApiClientFactory
from lusidjam.refreshing_token import RefreshingToken
from lusidtools.cocoon.cocoon import load_from_data_frame
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.cocoon.seed_sample_data import seed_data
from lusidtools.cocoon.utilities import create_scope_id
from lusidtools.cocoon.cocoon_printer import (
    format_instruments_response,
    format_portfolios_response,
    format_transactions_response,
    format_quotes_response,
    format_holdings_response,
)

from collections import defaultdict
import pandas as pd
import numpy as np
import json
import openpyxl
import os

pd.set_option('display.max_columns', None)

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename = secrets_path,
    app_name="LusidJupyterNotebook")

print ('LUSID Environment Initialised')
print ('API Version: ', api_factory.build(lusid.api.ApplicationMetadataApi).get_lusid_versions().build_version)

LUSID Environment Initialised
API Version:  0.0.1.0


Define scope, portfolio and some other variables used in the example:

In [2]:
orders_scope = "combining_fo_and_mo_scope"
orders_portfolio = "us_long_live"
ibor_scope = "middle_office"
ibor_portfolio = "us_long"
strategy_shk = "strategy"
executed_price_code = "executed_price"
execution_price_property = f"Allocation/{orders_scope}/{executed_price_code}"
allocation_strategy_property = f"Allocation/{orders_scope}/{strategy_shk}"
pf_created_date = "2020-01-01T00:00:00+00:00"

## Load an Instrument Master into LUSID

In [3]:
instr_df = pd.read_csv("data/multiplecurrencies-instruments.csv")
display(instr_df)

,instrument_name,client_internal,currency,isin,figi,exchange_code,country_issue,ticker,market_sector,security_type,coupon
0,Amazon_Nasdaq_AMZN,imd_34634534,USD,US0231351067,BBG000BVPXP1,UN,united_states_america,AMZN,equity,common_stock,NaN
1,Apple_Nasdaq_AAPL,imd_35345345,USD,US0378331005,BBG000B9XVV8,UN,united_states_america,AAPL,equity,common_stock,NaN
2,USTreasury_2.00_2021,imd_34535347,USD,US912828U816,BBG00FN3B5K8,BERLIN,united_states_america,T 2 12/31/21,govt,us_government,2.000
3,USTreasury_6.875_2025,imd_34534539,USD,US912810EV62,BBG000DQQNJ8,NEW YORK,united_states_america,T 6.875 08/15/25,govt,us_government,6.875
4,ExpressScripts_NYSE_ESRX,imd_34352311,USD,US30219G1085,BBG000C16621,UN,united_states_america,ESRX,equity,common_stock,NaN
5,TrinityIndustries_NYSE_TRN,imd_34235200,USD,US8965221091,BBG000BVL406,UN,united_states_america,TRN,equity,common_stock,NaN
6,Trex_NYSE_TREX,imd_32423956,USD,US89531P1057,BBG000BTGM43,UN,united_states_america,TREX,equity,common_stock,NaN
7,Cigna_NYSE_CI,imd_32452391,USD,US1255091092,BBG00KXXK940,UN,united_states_america,CI,equity,common_stock,NaN
8,Arcosa_NYSE_ACA,imd_23423409,USD,US0396531008,BBG00JGMWFQ5,UN,united_states_america,ACA,equity,common_stock,NaN


In [4]:
instrument_mapping = {
    "identifier_mapping": {
        "ClientInternal": "client_internal",
        "Isin": "isin",
        "Figi": "figi",
    },
    "required": {
        "name": "instrument_name"
    },
}

In [5]:
result = load_from_data_frame(
    api_factory=api_factory,
    scope=ibor_scope,
    data_frame=instr_df,
    mapping_required=instrument_mapping["required"],
    mapping_optional={},
    file_type="instruments",
    identifier_mapping=instrument_mapping["identifier_mapping"],
)

succ, failed, errors = format_instruments_response(result)
pd.DataFrame(data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}])

,success,failed,errors
0,9,0,0


## Create LUSID Properties

Create the properties for the sub-holding keys and executed price on the allocations

In [6]:
properties_api = api_factory.build(la.PropertyDefinitionsApi)

def create_property_definition(domain, scope, code, data_type):
    try:
        properties_api.create_property_definition(
            create_property_definition_request=lm.CreatePropertyDefinitionRequest(
                domain=domain,
                scope=scope,
                code=code,
                display_name=code,
                life_time="Perpetual",
                value_required=False,
                data_type_id=lm.resource_id.ResourceId(scope="system", code=data_type)
            )
        )
    except ApiException as e:
        detail = json.loads(e.body)
        if detail["code"] != 124:  # 'PropertyAlreadyExists'
            raise e

In [7]:
# sub-holding keys
create_property_definition("Transaction", ibor_scope, strategy_shk, "string")

# allocation execution price
create_property_definition("Allocation", orders_scope, executed_price_code, "number")

# allocation strategy name
create_property_definition("Allocation", orders_scope, strategy_shk, "string")

## Upload a Portfolio into LUSID

In [8]:
def create_portfolio(scope, portfolio_code, name):

    pf_df = pd.DataFrame(data=[
        {"portfolio_code": portfolio_code, "portfolio_name": name},
    ])
    
    portfolio_mapping = {
        "required": {
            "code": "portfolio_code",
            "display_name": "portfolio_name",
            "base_currency": "$USD",
        },
        "optional": {
            "created": f"${pf_created_date}"
        },
    }
    
    result = load_from_data_frame(
        api_factory=api_factory,
        scope=scope,
        data_frame=pf_df,
        mapping_required=portfolio_mapping["required"],
        mapping_optional=portfolio_mapping["optional"],
        file_type="portfolios",
        sub_holding_keys=[strategy_shk],
    )

    succ, failed = format_portfolios_response(result)
    display(failed)
    display(pd.DataFrame(data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}])) 

Create the main portfolio

In [9]:
create_portfolio(ibor_scope, ibor_portfolio, "ibor")

,error_items,status


,success,failed,errors
0,1,0,0


## Create a Derived portfolio to hold intraday activity

Create a derived portfolio to capture the executed orders to give us a front office live postion view

In [10]:
derivedpf_api = la.DerivedTransactionPortfoliosApi(api_factory.build(la.DerivedTransactionPortfoliosApi))

try:
    dp_request = lm.CreateDerivedTransactionPortfolioRequest(
        code="us_long_live",
        display_name="us_long_live",
        parent_portfolio_id=lm.ResourceId(ibor_scope, ibor_portfolio),
        created=pf_created_date,    
    )

    dp_result = derivedpf_api.create_derived_portfolio(scope=ibor_scope, create_derived_transaction_portfolio_request=dp_request)
except ApiException as e:
    detail = json.loads(e.body)
    if detail["code"] != 112:  # 'PortfolioWithIdAlreadyExists'
        raise e

## Set initial holdings in the parent portfolio

In [11]:
hldgs_df = pd.read_csv("data/initial_holdings.csv")
display(hldgs_df)

,instrument_name,client_internal,isin,figi,quantity,unit_cost,total_cost,currency,strategy
0,Amazon_Nasdaq_AMZN,imd_34634534,US0231351067,BBG000BVPXP1,5000,1550.00,7750000.00,USD,Tech
1,Apple_Nasdaq_AAPL,imd_35345345,US0378331005,BBG000B9XVV8,49567,190.00,9417730.00,USD,Tech
2,USTreasury_6.875_2025,imd_34534539,US912810EV62,BBG000DQQNJ8,98444,140.98,13878635.12,USD,Balanced
3,Apple_Nasdaq_AAPL,imd_35345375,US0378331005,BBG000B9XVV8,350000,190.00,9417730.00,USD,Balanced
4,USTreasury_2.00_2021,imd_34535363,US912828U816,BBG00FN3B5K8,1433,99.25,12063142.75,USD,Balanced


In [12]:
holdings_mapping = {
    "required":{
        "code": f"${ibor_portfolio}",
        "effective_at": "$2020-05-01",
        "tax_lots.units": "quantity"
    },
    "identifier_mapping": {
        "Figi": "figi"
    },
    "optional": {
        "tax_lots.cost.amount": "total_cost",
        "tax_lots.cost.currency": "currency",
        "tax_lots.price": "unit_cost"
    }
}

In [13]:
result = load_from_data_frame(
    api_factory=api_factory,
    scope=ibor_scope,
    data_frame=hldgs_df,
    mapping_required=holdings_mapping["required"],
    mapping_optional=holdings_mapping["optional"],
    identifier_mapping=holdings_mapping["identifier_mapping"],
    sub_holding_keys=[strategy_shk],
    file_type="holdings"
)

succ, failed = format_holdings_response(result)
pd.DataFrame(data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}])

,success,failed,errors
0,1,0,0


## Check the initial portfolio holdings

In [14]:
executed_holdings = api_factory.build(la.TransactionPortfoliosApi).get_holdings_with_orders(scope=ibor_scope, code=ibor_portfolio, property_keys=["Instrument/default/Name"])

lusid_response_to_data_frame(executed_holdings, rename_properties=True).sort_values(by='strategy(middle_office-SubHoldingKeys)')

,instrument_uid,strategy(middle_office-SubHoldingKeys),Name(default-Properties),SourcePortfolioId(default-Properties),SourcePortfolioScope(default-Properties),holding_type,units,settled_units,cost.amount,cost.currency,cost_portfolio_ccy.amount,cost_portfolio_ccy.currency,currency
2,LUID_00003D4Y,Balanced,Apple_Nasdaq_AAPL,us_long,middle_office,P,350000.0,350000.0,9417730.00,USD,0.0,USD,USD
3,LUID_00003D4X,Balanced,USTreasury_6.875_2025,us_long,middle_office,P,98444.0,98444.0,13878635.12,USD,0.0,USD,USD
4,LUID_00003D51,Balanced,USTreasury_2.00_2021,us_long,middle_office,P,1433.0,1433.0,12063142.75,USD,0.0,USD,USD
0,LUID_00003D4Z,Tech,Amazon_Nasdaq_AMZN,us_long,middle_office,P,5000.0,5000.0,7750000.00,USD,0.0,USD,USD
1,LUID_00003D4Y,Tech,Apple_Nasdaq_AAPL,us_long,middle_office,P,49567.0,49567.0,9417730.00,USD,0.0,USD,USD


## Post orders into LUSID

In this section, we post some [orders](https://support.finbourne.com/how-does-lusid-support-the-trade-lifecycle) into LUSID. 

In [25]:
orders_df = pd.read_csv('data/orders.csv')
orders_df

,portfolio,instrument_name,client_internal,isin,figi,quantity,price,currency,order_id,side,type,strategy,state
0,us_long,USTreasury_2.00_2021,imd_34535347,US912828U816,BBG00FN3B5K8,121543,99.25,USD,ORD001,buy,limit,Balanced,new
1,us_long,USTreasury_2.00_2021,imd_34535347,US912828U816,BBG00FN3B5K8,121,99.30,USD,ORD001,buy,limit,Balanced,new
2,us_long,Amazon_Nasdaq_AMZN,imd_346345343,US0231351067,BBG000BVPXP1,500,1400.00,USD,ORD002,sell,limit,Tech,new
3,us_long,Amazon_Nasdaq_AMZN,imd_346345343,US0231351067,BBG000BVPXP1,4000,1550.00,USD,ORD003,buy,limit,Balanced,new
4,us_long,USTreasury_6.875_2025,imd_34534512,US912810EV62,BBG000DQQNJ8,5643,140.98,USD,ORD004,buy,limit,Balanced,new
5,us_long,USTreasury_6.875_2025,imd_34534539,US912810EV62,BBG000DQQNJ8,32000,150.00,USD,ORD005,sell,limit,Balanced,new


In [26]:
order_requests = defaultdict(list)
order_sets = defaultdict(list)

for index, order in orders_df.iterrows():
    
    portfolio = order['portfolio']

    request=models.OrderSetRequest(
        order_requests=[models.OrderRequest(
            id=models.ResourceId(
                scope=orders_scope,
                code=order['order_id']
            ),
            quantity=order['quantity'],
            side=order['side'],
            instrument_identifiers={
                'Instrument/default/Figi': order['figi']
            },
            properties={},
            portfolio_id=models.ResourceId(
                scope=ibor_scope,
                code=ibor_portfolio
            ),
            state="new",
            type=order['type'],
            price=models.CurrencyAndAmount(
                        amount=order['quantity'] * order['price'],
                        currency=order['currency'])
        )]           
    )

    response = api_factory.build(lusid.api.OrdersApi).upsert_orders(
        order_set_request=request
    )
    
    display(lusid_response_to_data_frame(response))

,properties,version.effective_from,version.as_at_date,instrument_identifiers.Instrument/default/Figi,quantity,side,order_book_id.scope,order_book_id.code,portfolio_id.scope,portfolio_id.code,id.scope,id.code,lusid_instrument_id,state,type,date,price.amount,price.currency,order_instruction_id.scope,order_instruction_id.code,package_id.scope,package_id.code
0,{},0001-01-01 00:00:00+00:00,2021-09-30 21:11:59.764854+00:00,BBG00FN3B5K8,121543,buy,UnknownScope,UnknownOrderBook,middle_office,us_long,combining_fo_and_mo_scope,ORD001,LUID_00003D51,new,limit,2021-09-30 21:11:59.755650+00:00,12063142.75,USD,UnknownScope,UnknownOrderInstruction,UnknownScope,UnknownPackage


,properties,version.effective_from,version.as_at_date,instrument_identifiers.Instrument/default/Figi,quantity,side,order_book_id.scope,order_book_id.code,portfolio_id.scope,portfolio_id.code,id.scope,id.code,lusid_instrument_id,state,type,date,price.amount,price.currency,order_instruction_id.scope,order_instruction_id.code,package_id.scope,package_id.code
0,{},0001-01-01 00:00:00+00:00,2021-09-30 21:11:59.808617+00:00,BBG00FN3B5K8,121,buy,UnknownScope,UnknownOrderBook,middle_office,us_long,combining_fo_and_mo_scope,ORD001,LUID_00003D51,new,limit,2021-09-30 21:11:59.797560+00:00,12015.3,USD,UnknownScope,UnknownOrderInstruction,UnknownScope,UnknownPackage


,properties,version.effective_from,version.as_at_date,instrument_identifiers.Instrument/default/Figi,quantity,side,order_book_id.scope,order_book_id.code,portfolio_id.scope,portfolio_id.code,id.scope,id.code,lusid_instrument_id,state,type,date,price.amount,price.currency,order_instruction_id.scope,order_instruction_id.code,package_id.scope,package_id.code
0,{},0001-01-01 00:00:00+00:00,2021-09-30 21:11:59.853916+00:00,BBG000BVPXP1,500,sell,UnknownScope,UnknownOrderBook,middle_office,us_long,combining_fo_and_mo_scope,ORD002,LUID_00003D4Z,new,limit,2021-09-30 21:11:59.844882+00:00,700000.0,USD,UnknownScope,UnknownOrderInstruction,UnknownScope,UnknownPackage


,properties,version.effective_from,version.as_at_date,instrument_identifiers.Instrument/default/Figi,quantity,side,order_book_id.scope,order_book_id.code,portfolio_id.scope,portfolio_id.code,id.scope,id.code,lusid_instrument_id,state,type,date,price.amount,price.currency,order_instruction_id.scope,order_instruction_id.code,package_id.scope,package_id.code
0,{},0001-01-01 00:00:00+00:00,2021-09-30 21:11:59.893043+00:00,BBG000BVPXP1,4000,buy,UnknownScope,UnknownOrderBook,middle_office,us_long,combining_fo_and_mo_scope,ORD003,LUID_00003D4Z,new,limit,2021-09-30 21:11:59.886044+00:00,6200000.0,USD,UnknownScope,UnknownOrderInstruction,UnknownScope,UnknownPackage


,properties,version.effective_from,version.as_at_date,instrument_identifiers.Instrument/default/Figi,quantity,side,order_book_id.scope,order_book_id.code,portfolio_id.scope,portfolio_id.code,id.scope,id.code,lusid_instrument_id,state,type,date,price.amount,price.currency,order_instruction_id.scope,order_instruction_id.code,package_id.scope,package_id.code
0,{},0001-01-01 00:00:00+00:00,2021-09-30 21:11:59.929871+00:00,BBG000DQQNJ8,5643,buy,UnknownScope,UnknownOrderBook,middle_office,us_long,combining_fo_and_mo_scope,ORD004,LUID_00003D4X,new,limit,2021-09-30 21:11:59.922474+00:00,795550.14,USD,UnknownScope,UnknownOrderInstruction,UnknownScope,UnknownPackage


,properties,version.effective_from,version.as_at_date,instrument_identifiers.Instrument/default/Figi,quantity,side,order_book_id.scope,order_book_id.code,portfolio_id.scope,portfolio_id.code,id.scope,id.code,lusid_instrument_id,state,type,date,price.amount,price.currency,order_instruction_id.scope,order_instruction_id.code,package_id.scope,package_id.code
0,{},0001-01-01 00:00:00+00:00,2021-09-30 21:11:59.970479+00:00,BBG000DQQNJ8,32000,sell,UnknownScope,UnknownOrderBook,middle_office,us_long,combining_fo_and_mo_scope,ORD005,LUID_00003D4X,new,limit,2021-09-30 21:11:59.961673+00:00,4800000.0,USD,UnknownScope,UnknownOrderInstruction,UnknownScope,UnknownPackage


## Book Allocations against the orders

[Allocations](https://support.finbourne.com/what-is-an-allocation-in-lusid) are then booked off the back of these orders.

In [34]:
allocations_df = pd.read_csv('data/allocations.csv')
allocations_df

,portfolio,instrument_name,client_internal,isin,figi,quantity,price,currency,allocation_id,originating_order,state,side,strategy
0,us_long,USTreasury_2.00_2021,imd_34535347,US912828U816,BBG00FN3B5K8,121543,80,USD,ALLOC001,ORD001,new,buy,Balanced
1,us_long,Amazon_Nasdaq_AMZN,imd_346345343,US0231351067,BBG000BVPXP1,500,1600,USD,ALLOC002,ORD002,new,sell,Tech
2,us_long,Amazon_Nasdaq_AMZN,imd_346345343,US0231351067,BBG000BVPXP1,4000,1600,USD,ALLOC003,ORD003,new,buy,Balanced
3,us_long,USTreasury_6.875_2025,imd_34534512,US912810EV62,BBG000DQQNJ8,5643,150,USD,ALLOC004,ORD004,new,buy,Balanced
4,us_long,USTreasury_6.875_2025,imd_34534539,US912810EV62,BBG000DQQNJ8,32000,150,USD,ALLOC005,ORD005,new,sell,Balanced


In [35]:
allocation_requests = defaultdict(list)
allocation_sets = defaultdict(list)

for index, allocation in allocations_df.iterrows():
    
    portfolio = allocation['portfolio']

    allocation_requests[portfolio].append(
        models.AllocationRequest(
            id=models.ResourceId(
                scope=orders_scope,
                code=allocation['allocation_id']
            ),
            allocated_order_id=models.ResourceId(
                scope=orders_scope,
                code=allocation['originating_order']
            ),
            quantity=allocation['quantity'],
            instrument_identifiers={
                'Instrument/default/Figi': allocation['figi']
            },
            properties={
                execution_price_property: lm.PerpetualProperty(
                    key=execution_price_property,
                    value=lm.PropertyValue(metric_value=lm.MetricValue(value=allocation["price"]))
                ),
                allocation_strategy_property: lm.PerpetualProperty(
                    key=allocation_strategy_property,
                    value=lm.PropertyValue(label_value=allocation['strategy'])
                )
            },
            portfolio_id=models.ResourceId(
                scope=ibor_scope,
                code=ibor_portfolio
            ),
            price=models.CurrencyAndAmount(
                amount=allocation['quantity'] * allocation['price'],
                currency=allocation['currency']),
            state=allocation['state'],
            side=allocation['side']
        )
    )
    
for allocation_portfolio in allocation_requests:    
    allocation_sets[allocation_portfolio] = models.AllocationSetRequest(
        allocation_requests=allocation_requests[allocation_portfolio]
    )

for allocation_portfolio in allocation_sets:
    response = api_factory.build(lusid.api.AllocationsApi).upsert_allocations(
        allocation_set_request=allocation_sets[allocation_portfolio]
    )
    
    display(lusid_response_to_data_frame(response))

,id.scope,id.code,allocated_order_id.scope,allocated_order_id.code,portfolio_id.scope,portfolio_id.code,quantity,instrument_identifiers.Instrument/default/Figi,version.effective_from,version.as_at_date,properties.Allocation/combining_fo_and_mo_scope/strategy.key,properties.Allocation/combining_fo_and_mo_scope/strategy.value.label_value,properties.Allocation/combining_fo_and_mo_scope/executed_price.key,properties.Allocation/combining_fo_and_mo_scope/executed_price.value.metric_value.value,lusid_instrument_id,placement_ids,state,side,type,settlement_date,date,price.amount,price.currency,settlement_currency,settlement_currency_fx_rate,counterparty
0,combining_fo_and_mo_scope,ALLOC001,combining_fo_and_mo_scope,ORD001,middle_office,us_long,121543,BBG00FN3B5K8,0001-01-01 00:00:00+00:00,2021-09-30 21:17:14.715959+00:00,Allocation/combining_fo_and_mo_scope/strategy,Balanced,Allocation/combining_fo_and_mo_scope/executed_...,80.0,LUID_00003D51,[],new,buy,UnknownType,0001-01-01 00:00:00+00:00,0001-01-01 00:00:00+00:00,9723440.0,USD,CCY_ZZZ,0.0,UnknownCounterparty
1,combining_fo_and_mo_scope,ALLOC003,combining_fo_and_mo_scope,ORD003,middle_office,us_long,4000,BBG000BVPXP1,0001-01-01 00:00:00+00:00,2021-09-30 21:17:14.715959+00:00,Allocation/combining_fo_and_mo_scope/strategy,Balanced,Allocation/combining_fo_and_mo_scope/executed_...,1600.0,LUID_00003D4Z,[],new,buy,UnknownType,0001-01-01 00:00:00+00:00,0001-01-01 00:00:00+00:00,6400000.0,USD,CCY_ZZZ,0.0,UnknownCounterparty
2,combining_fo_and_mo_scope,ALLOC005,combining_fo_and_mo_scope,ORD005,middle_office,us_long,32000,BBG000DQQNJ8,0001-01-01 00:00:00+00:00,2021-09-30 21:17:14.715959+00:00,Allocation/combining_fo_and_mo_scope/strategy,Balanced,Allocation/combining_fo_and_mo_scope/executed_...,150.0,LUID_00003D4X,[],new,sell,UnknownType,0001-01-01 00:00:00+00:00,0001-01-01 00:00:00+00:00,4800000.0,USD,CCY_ZZZ,0.0,UnknownCounterparty
3,combining_fo_and_mo_scope,ALLOC002,combining_fo_and_mo_scope,ORD002,middle_office,us_long,500,BBG000BVPXP1,0001-01-01 00:00:00+00:00,2021-09-30 21:17:14.715959+00:00,Allocation/combining_fo_and_mo_scope/strategy,Tech,Allocation/combining_fo_and_mo_scope/executed_...,1600.0,LUID_00003D4Z,[],new,sell,UnknownType,0001-01-01 00:00:00+00:00,0001-01-01 00:00:00+00:00,800000.0,USD,CCY_ZZZ,0.0,UnknownCounterparty
4,combining_fo_and_mo_scope,ALLOC004,combining_fo_and_mo_scope,ORD004,middle_office,us_long,5643,BBG000DQQNJ8,0001-01-01 00:00:00+00:00,2021-09-30 21:17:14.715959+00:00,Allocation/combining_fo_and_mo_scope/strategy,Balanced,Allocation/combining_fo_and_mo_scope/executed_...,150.0,LUID_00003D4X,[],new,buy,UnknownType,0001-01-01 00:00:00+00:00,0001-01-01 00:00:00+00:00,846450.0,USD,CCY_ZZZ,0.0,UnknownCounterparty


## Check the portfolio holdings with orders

In [36]:
executed_holdings = api_factory.build(la.TransactionPortfoliosApi).get_holdings_with_orders(scope=ibor_scope, code=ibor_portfolio, property_keys=["Instrument/default/Name"])

lusid_response_to_data_frame(executed_holdings, rename_properties=True).sort_values(by='strategy(middle_office-SubHoldingKeys)')

,instrument_uid,strategy(middle_office-SubHoldingKeys),Name(default-Properties),SourcePortfolioId(default-Properties),SourcePortfolioScope(default-Properties),holding_type,units,settled_units,cost.amount,cost.currency,cost_portfolio_ccy.amount,cost_portfolio_ccy.currency,currency,sub_holding_keys,transaction.transaction_id,transaction.type,transaction.instrument_identifiers.Instrument/default/Figi,transaction.instrument_uid,transaction.transaction_date,transaction.settlement_date,transaction.units,transaction.transaction_price.price,transaction.transaction_price.type,transaction.total_consideration.amount,transaction.total_consideration.currency,transaction.exchange_rate,transaction.transaction_currency,transaction.properties.Transaction/default/TradeToPortfolioRate.key,transaction.properties.Transaction/default/TradeToPortfolioRate.value.metric_value.value,transaction.properties.Transaction/default/TradeToPortfolioRate.value.metric_value.unit,transaction.source,transaction.entry_date_time
2,LUID_00003D4Y,Balanced,Apple_Nasdaq_AAPL,us_long,middle_office,P,3.500000e+05,350000.0,9.417730e+06,USD,0.000000e+00,USD,USD,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
3,LUID_00003D4X,Balanced,USTreasury_6.875_2025,us_long,middle_office,P,9.844400e+04,98444.0,1.387864e+07,USD,0.000000e+00,USD,USD,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
4,LUID_00003D51,Balanced,USTreasury_2.00_2021,us_long,middle_office,P,1.433000e+03,1433.0,1.206314e+07,USD,0.000000e+00,USD,USD,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
0,LUID_00003D4Z,Tech,Amazon_Nasdaq_AMZN,us_long,middle_office,P,5.000000e+03,5000.0,7.750000e+06,USD,0.000000e+00,USD,USD,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,LUID_00003D4Y,Tech,Apple_Nasdaq_AAPL,us_long,middle_office,P,4.956700e+04,49567.0,9.417730e+06,USD,0.000000e+00,USD,USD,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
5,LUID_00003D4X,NaN,USTreasury_6.875_2025,us_long,middle_office,O,0.000000e+00,0.0,0.000000e+00,USD,0.000000e+00,USD,USD,{},VirtualTrade-Order-combining_fo_and_mo_scope/O...,Sell,BBG000DQQNJ8,LUID_00003D4X,2021-09-30 21:17:20.018232+00:00,2021-10-03 21:17:20.018232+00:00,0.0,4800000.00,Price,0.000000e+00,USD,1.0,USD,Transaction/default/TradeToPortfolioRate,1.0,,,2021-09-30 21:17:20.018232+00:00
6,CCY_USD,NaN,UNITED STATES DOLLAR,us_long,middle_office,OC,0.000000e+00,0.0,0.000000e+00,USD,0.000000e+00,USD,USD,{},VirtualTrade-Order-combining_fo_and_mo_scope/O...,Sell,BBG000DQQNJ8,LUID_00003D4X,2021-09-30 21:17:20.018232+00:00,2021-10-03 21:17:20.018232+00:00,0.0,4800000.00,Price,0.000000e+00,USD,1.0,USD,Transaction/default/TradeToPortfolioRate,1.0,,,2021-09-30 21:17:20.018232+00:00
7,LUID_00003D4X,NaN,USTreasury_6.875_2025,us_long,middle_office,O,0.000000e+00,0.0,0.000000e+00,USD,0.000000e+00,USD,USD,{},VirtualTrade-Order-combining_fo_and_mo_scope/O...,Buy,BBG000DQQNJ8,LUID_00003D4X,2021-09-30 21:17:20.018232+00:00,2021-10-03 21:17:20.018232+00:00,0.0,795550.14,Price,0.000000e+00,USD,1.0,USD,Transaction/default/TradeToPortfolioRate,1.0,,,2021-09-30 21:17:20.018232+00:00
8,CCY_USD,NaN,UNITED STATES DOLLAR,us_long,middle_office,OC,0.000000e+00,0.0,0.000000e+00,USD,0.000000e+00,USD,USD,{},VirtualTrade-Order-combining_fo_and_mo_scope/O...,Buy,BBG000DQQNJ8,LUID_00003D4X,2021-09-30 21:17:20.018232+00:00,2021-10-03 21:17:20.018232+00:00,0.0,795550.14,Price,0.000000e+00,USD,1.0,USD,Transaction/default/TradeToPortfolioRate,1.0,,,2021-09-30 21:17:20.018232+00:00
9,LUID_00003D4Z,NaN,Amazon_Nasdaq_AMZN,us_long,middle_office,O,0.000000e+00,0.0,0.000000e+00,USD,0.000000e+00,USD,USD,{},VirtualTrade-Order-combining_fo_and_mo_scope/O...,Sell,BBG000BVPXP1,LUID_00003D4Z,2021-09-30 21:17:20.018232+00:00,2021-10-03 21:17:20.018232+00:00,0.0,700000.00,Price,0.000000e+00,USD,1.0,USD,Transaction/default/TradeToPortfolioRate,1.0,,,2021-09-30 21:17:20.018232+00:00
